**1. Nuget Downloads**

In [ ]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22504.6 Microsoft.ML, 2.0.0-preview.22313.1

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

ExplainCode 
 This extension generates Sequence diagrams from csharp code using Mermaid kernel.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;

using System.IO;
using System.Text;

2. Data Models

In [ ]:
public class PenguinData
{
    [LoadColumn(0)]
    public string Label {get;set;}

    [LoadColumn(1)]
    public string Island {get;set;}

    [LoadColumn(2)]
    public float CulmenLength {get;set;}

    [LoadColumn(3)]
    public float CulmenDepth {get;set;}

    [LoadColumn(4)]
    public float FilperLength {get;set;}

    [LoadColumn(5)]
    public float BodyMass {get;set;}

    [LoadColumn(6)]
    public string Sex {get;set;}
}

public class PenguinPrediction
{
    [ColumnName("PredictedLabel")]
    public string PredictedLabel {get;set;}
}

3. Load and Split Data

In [ ]:
var mlContext = new MLContext();
var trainingDataView = mlContext.Data.LoadFromTextFile<PenguinData>("../../ML.Net.Training/Datasets/penguins_size.csv", hasHeader: true, separatorChar: ',');
var dataSplit = mlContext.Data.TrainTestSplit(trainingDataView, testFraction: 0.3);

In [ ]:
dataSplit.TestSet.ToTabularDataResource().Display();

4. Train the Model

In [ ]:
var model = mlContext.MulticlassClassification.Trainers.SdcaNonCalibrated(labelColumnName: "Label", featureColumnName: "Features");

var pipeline = mlContext.Transforms.Conversion.MapValueToKey(inputColumnName: nameof(PenguinData.Label), outputColumnName: "Label")
                .Append(mlContext.Transforms.Text.FeaturizeText(inputColumnName: "Sex", outputColumnName: "SexFeaturized"))
                .Append(mlContext.Transforms.Text.FeaturizeText(inputColumnName: "Island", outputColumnName: "IslandFeaturized"))
                .Append(mlContext.Transforms.Concatenate("Features", 
                                                         "IslandFeaturized",
                                                         nameof(PenguinData.CulmenLength),
                                                         nameof(PenguinData.CulmenDepth),
                                                         nameof(PenguinData.BodyMass),
                                                         nameof(PenguinData.FilperLength),
                                                         "SexFeaturized"))
                .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
                .Append(model)
                .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

5. Evaluate Model

In [ ]:
var testSetTransform = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.MulticlassClassification.Evaluate(testSetTransform);

In [ ]:
metrics.MacroAccuracy

0.991869918699187

6. Save Model

In [ ]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./MultiClassificationModel.mdl");

7. Load and Predict

In [ ]:
var newSample = new PenguinData
{
    Island = "Torgersen",
    CulmenLength = 18.7f,
    CulmenDepth = 39.3f,
    FilperLength = 180,
    BodyMass = 3700,
    Sex = "MALE"
};

using (var stream = new FileStream("./MultiClassificationModel.mdl", FileMode.Open, FileAccess.Read, FileShare.Read))
{
    var loadedModel = mlContext.Model.Load(stream, out _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<PenguinData, PenguinPrediction>(loadedModel);

    var result = predictionEngine.Predict(newSample);

    Console.WriteLine("Prediction: " + result.PredictedLabel);
}

Prediction: Adelie
